<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/CreateGSPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключение данных


In [1]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime

# worksheet = gc.open('План работ Договор Адамант').sheet1
worksheet = gc.open('План работ Договор Адамант')

# get_all_values gives a list of rows.
# rows = worksheet.get_all_values()
# print(rows)

# # Convert to a DataFrame and render.
# pd.DataFrame.from_records(rows)

* данный финансового плана импортируются из АСС, папка соответсвующего договора TODO
* объем по контракту необходимо формировать из бим-модели, либо подгружать из РД, через процедуру проверки по ППР TODO

In [2]:
import numpy as np
fin = worksheet.worksheet("FINANCE PLAN")
fin = fin.get_all_values()
fin = np.array(fin)
fin =  pd.DataFrame(fin[5:, :7])
fin

,0,1,2,3,4,5,6
0,VZO_POS1_POD_MOB_.,Мобилизация,10.10.2022,10.05.2024,,,
1,VZO_POS1_POD_VRD_DP17.01,Временные дороги из дорожных плит (3х1.75),26.10.2022,18.02.2023,"3 646,89 ₽",м2,300
2,VZO_POS1_POD_VRS_ES0.01,Временные электросети - установленные на начал...,23.11.2022,12.12.2022,"333 333,34 ₽",комплект,2
3,VZO_POS1_POD_VRS_VK0.01,Временный водопровод и канализация (с подогрев...,19.10.2022,14.11.2022,"6 666 666,67 ₽",комплект,1
4,VZO_POS1_POD_VRD_DP17.02,Устройство основания под бытовые помещения (по...,16.10.2022,09.11.2022,"3 669,83 ₽",м2,200
...,...,...,...,...,...,...,...
368,BOT_GP___.,Работы по благоустройству,18.11.2022,05.04.2024,,,
369,BOT_GP,Выдача рабочего проекта по разделу,18.11.2022,18.11.2022,,,
370,_EXP___.,Ввод объекта в эксплуатацию,14.03.2024,06.06.2024,,,
371,_ZOS___.,ЗОС,14.03.2024,24.04.2024,,,


In [3]:
fin.rename(columns = {0:'id', 
                       1:'WORK',2:'START', 3:'FINISH', 4:'PRICE for ONE', 5:'UNITS OF MEASUREMENT',6:'VALUE CONTRACT'}, 
            inplace = True)

In [4]:
fin = fin.set_index("id")
fin = fin[fin.index > '']

Приводим строковые значения к числовым 

In [5]:
fin['PRICE for ONE'] = fin['PRICE for ONE'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
fin['VALUE CONTRACT'] = fin['VALUE CONTRACT'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))

Приводим значения дат в формат date

In [6]:
fin['START'] = pd.to_datetime(fin['START'])   
fin['FINISH']= pd.to_datetime(fin['FINISH'])   

Завершаем предобработку таблицы - заменяем все значения Nan на "0"

In [7]:
fin = fin.fillna(0)

# Формирование листа FINANCE

Задача: на основе данных из условий договора и данных по планируемому и фактическому выполнению (сейчас данные по факту просто берем из таблицы ПЛАН)

* расчет общей стоимости по контракту
* формирование плановой стоимости работ на соответсвующую дату: за период
* формирование плановой стоимости работ на соответсвующую дату: накопительная сумма
* формирование фактической стоимости работ на соответсвующую дату: за период
* формирование фактической стоимости работ на соответсвующую дату: накопительная сумма
* внесение корректировок и обновлений при формировании ДС

❗ сравнение с нашей базой расценок 




In [8]:
fin['TOTAL PRICE'] = fin['PRICE for ONE'] * fin['VALUE CONTRACT']

# Предобработка листа ГРАФИК РАБОТ

Подключаем лист ГРАФИК РАБОТ

In [9]:
graf = worksheet.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf =  pd.DataFrame(graf[1:, :])

In [10]:
graf_name = graf[[0,1]][2:]
graf_name.rename(columns={0: 'id', 1: 'WORK'}, inplace=True) 
graf_name = graf_name[graf_name['id'] != 0]
graf_name = pd.DataFrame(graf_name['WORK'].values, index=graf_name['id'], columns=['WORK'])

Предобработка данных в таблице:
* приведение данных в необходимый формат
* корректировка наименования столбцов и индексов

In [11]:
graf.drop(graf.index[1:10], axis=1, inplace=True)

Формируем два списка для мультииндексов

In [12]:
newColumnsIndex = [pd.to_datetime(pd.Series(graf.iloc[0].values[1:]).apply(lambda x: str(x)), format='%d.%m.%Y'), graf.iloc[1][1:].values]

In [13]:
graf.rename(columns={0: 'id'}, inplace=True)

In [14]:
graf = graf.set_index("id")

In [15]:
graf = pd.DataFrame(graf.values, index=graf.index, columns=newColumnsIndex)

In [16]:
graf.drop(graf.index[:2], axis=0, inplace=True)

In [17]:
for i in graf.columns:
    graf[i] = graf[i].map(lambda x: pd.to_numeric(re.sub("[^0-9 +,]", "", x).replace(',','.')))

In [18]:
graf.fillna(0, inplace=True)
graf.sort_index(axis=0, level=1, ascending=True)

2022-10-10      2022-10-11      2022-10-12       \
                              план факт       план факт       план факт   
id                                                                        
                               0.0  0.0        0.0  0.0        0.0  0.0   
                               0.0  0.0        0.0  0.0        0.0  0.0   
                               0.0  0.0        0.0  0.0        0.0  0.0   
                               0.0  0.0        0.0  0.0        0.0  0.0   
                               0.0  0.0        0.0  0.0        0.0  0.0   
...                            ...  ...        ...  ...        ...  ...   
VZO_POS1_POD_VRZ_MBT.01        0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRZ_WC.02         0.0  0.0        0.0  0.0        0.0  0.0   
_EXP___.                       0.0  0.0        0.0  0.0        0.0  0.0   
_RNV___.                       0.0  0.0        0.0  0.0        0.0  0.0   
_ZOS___.                       0.0  0.0        0.0  0.0        0.0  0.0   

                        2022-10-13      2022-10-14       ... 2023-02-23       \
                              план факт       план факт  ...       план факт   
id                                                       ...                   
                               0.0  0.0        0.0  0.0  ...        0.0  0.0   
                               0.0  0.0        0.0  0.0  ...        0.0  0.0   
                               0.0  0.0        0.0  0.0  ...        0.0  0.0   
                               0.0  0.0        0.0  0.0  ...        0.0  0.0   
                               0.0  0.0        0.0  0.0  ...        0.0  0.0   
...                            ...  ...        ...  ...  ...        ...  ...   
VZO_POS1_POD_VRZ_MBT.01        0.0  0.0        0.0  0.0  ...        0.0  0.0   
VZO_POS1_POD_VRZ_WC.02         0.0  0.0        0.0  0.0  ...        0.0  0.0   
_EXP___.                       0.0  0.0        0.0  0.0  ...        0.0  0.0   
_RNV___.                       0.0  0.0        0.0  0.0  ...        0.0  0.0   
_ZOS___.                       0.0  0.0        0.0  0.0  ...        0.0  0.0   

                        2023-02-24      2023-02-25      2023-02-26       \
                              план факт       план факт       план факт   
id                                                                        
                               0.0  0.0        0.0  0.0        0.0  0.0   
                               0.0  0.0        0.0  0.0        0.0  0.0   
                               0.0  0.0        0.0  0.0        0.0  0.0   
                               0.0  0.0        0.0  0.0        0.0  0.0   
                               0.0  0.0        0.0  0.0       47.3  0.0   
...                            ...  ...        ...  ...        ...  ...   
VZO_POS1_POD_VRZ_MBT.01        0.0  0.0        0.0  0.0        0.0  0.0   
VZO_POS1_POD_VRZ_WC.02         0.0  0.0        0.0  0.0        0.0  0.0   
_EXP___.                       0.0  0.0        0.0  0.0        0.0  0.0   
_RNV___.                       0.0  0.0        0.0  0.0        0.0  0.0   
_ZOS___.                       0.0  0.0        0.0  0.0        0.0  0.0   

                        2023-02-27       
                              план факт  
id                                       
                               0.0  0.0  
                               0.0  0.0  
                               0.0  0.0  
                               0.0  0.0  
                               0.0  0.0  
...                            ...  ...  
VZO_POS1_POD_VRZ_MBT.01        0.0  0.0  
VZO_POS1_POD_VRZ_WC.02         0.0  0.0  
_EXP___.                       0.0  0.0  
_RNV___.                       0.0  0.0  
_ZOS___.                       0.0  0.0  

[398 rows x 282 columns]

# ПЛАН/ФАКТ
* выполненных работ
* стоимости выполненных работ
* формирование КС


In [19]:
idx = pd.IndexSlice

In [20]:
plan = graf.loc[:, idx['2022-12-24':'2022-12-30','план']].sum(axis=1).values
total_plan = graf.loc[:, idx[:,'план']].sum(axis=1).values
plan = pd.Series(plan, index=graf.index, name='PLAN_WEEK')
total_plan = pd.Series(total_plan, index=graf.index, name='TOTAL_PLAN')

In [29]:
plan_next = graf.loc[:, idx['2022-12-31':'2023-01-09','план']].sum(axis=1).values
plan_next = pd.Series(plan_next, index=graf.index, name='PLAN_NEXT_WEEK')

In [22]:
fakt = graf.loc[:, idx['2022-12-24':'2022-12-30','факт']].sum(axis=1).values
fakt = pd.Series(fakt, index=graf.index, name='FAKT_WEEK')
total_fakt = graf.loc[:, idx[:,'факт']].sum(axis=1).values
total_fakt = pd.Series(total_fakt, index=graf.index, name='TOTAL_FAKT')

In [30]:
fakt_next = graf.loc[:, idx['2022-12-31':'2023-01-09','факт']].sum(axis=1).values
fakt_next = pd.Series(fakt_next, index=graf.index, name='FAKT_NEXT_WEEK')

In [31]:
planFakt = pd.concat([graf_name,total_plan, total_fakt,plan,fakt,plan_next,fakt_next], axis=1)

In [32]:
planFakt = planFakt[planFakt['PLAN_WEEK'] != 0]

# Лист PLAN_WEEK

TODO => chartWEEK, ед измерения, менять даты по дням, chart оплтат

In [ ]:
# worksheet.add_worksheet('PLAN_WEEK',planFakt.shape[0], planFakt.shape[1])

<Worksheet 'PLAN_WEEK_new' id:1267137160>

In [ ]:
# worksheet.worksheet('PLAN_WEEK').clear

In [33]:
worksheet.worksheet('PLAN_WEEK').update('A1:G300',[planFakt[2:].columns.values.tolist()]+planFakt.values.tolist())

{'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
 'updatedRange': 'PLAN_WEEK!A1:G44',
 'updatedRows': 44,
 'updatedColumns': 7,
 'updatedCells': 308}

In [ ]:
# worksheet.worksheet('PLAN_WEEK').append_rows([planFakt.columns.values.tolist()]+planFakt.values.tolist())

{'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
 'tableRange': 'PLAN_WEEK!A1:I33',
 'updates': {'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
  'updatedRange': 'PLAN_WEEK!A34:G66',
  'updatedRows': 33,
  'updatedColumns': 7,
  'updatedCells': 231}}

In [ ]:
dir(worksheet.worksheet('PLAN_WEEK'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_finder',
 '_properties',
 'acell',
 'add_cols',
 'add_rows',
 'append_row',
 'append_rows',
 'batch_get',
 'batch_update',
 'cell',
 'clear',
 'clear_basic_filter',
 'client',
 'col_count',
 'col_values',
 'delete_row',
 'delete_rows',
 'duplicate',
 'export',
 'find',
 'findall',
 'format',
 'freeze',
 'frozen_col_count',
 'frozen_row_count',
 'get',
 'get_all_records',
 'get_all_values',
 'id',
 'insert_row',
 'merge_cells',
 'range',
 'resize',
 'row_count',
 'row_values',
 'set_basic_filter',
 'sort',
 'spreadsheet',
 'title',
 'update',
 'update_acell',
 'update_cell',
 'update_cells',
 'update_index',
 'update_tit